In [3]:
%gui qt5
%load_ext tensorboard

from skimage import data
import napari
import imageio
import glob
from skimage.transform import rescale
import os
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
from torchvision import transforms
from torch.optim import Adam

QStandardPaths: XDG_RUNTIME_DIR points to non-existing path '/run/user/1043', please create it with 0700 permissions.


In [4]:
class DrosoMicroCT(Dataset):
    '''A dataset to load microCT scans of Drosophila and organ segmentations'''
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir  # the directory with all the training samples
        self.samples = os.listdir(os.path.join(root_dir,  'images')) # list the samples
        self.transform = transform    # transformations to apply to both inputs and targets
        #  transformations to apply just to inputs
        self.inp_transforms = transforms.Compose([transforms.Normalize([0.5], [0.5])])
        # self.inp_transforms = transforms.Compose([transforms.ToTensor(),transforms.Normalize([0.5], [0.5])])
        # transformations to apply just to targets
        #self.mask_transforms = torch.tensor()
        
        # get the total number of samples
    def __len__(self):
        return len(self.samples)
    
    # fetch the training sample given its index
    def __getitem__(self, idx):
        img_path = os.path.join(self.root_dir,
                                'images', self.samples[idx])
        image = imageio.volread(img_path)
        image = torch.tensor(image)
        image = self.inp_transforms(image)
        labels_path = os.path.join(self.root_dir, 'labels', self.samples[idx].replace('Rec', 'Rec_labels'))
        labels = imageio.volread(labels_path)
        labels = torch.tensor(labels)
        labels[labels == 4] = 3 #converts ovaries to same label
        image = torch.unsqueeze(image, dim=0)
        labels = torch.unsqueeze(labels, dim=0)
        if self.transform is not None:
            image, labels = self.transform([image, labels])
        return image.type(torch.FloatTensor), labels.type(torch.FloatTensor)
        #return image, labels

In [3]:
train_dataset = DrosoMicroCT('./Project5_Drosophila_microCT_segmentation/resize2/train')
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

val_data = DrosoMicroCT('./Project5_Drosophila_microCT_segmentation/resize2/val')
val_loader = DataLoader(val_data, batch_size=2)

In [5]:
class UNet(nn.Module):
    """ UNet implementation
    Arguments:
      in_channels: number of input channels
      out_channels: number of output channels
      final_activation: activation applied to the network output
    """
    
    # _conv_block and _upsampler are just helper functions to
    # construct the model.
    # encapsulating them like so also makes it easy to re-use
    # the model implementation with different architecture elements
    
    # Convolutional block for single layer of the decoder / encoder
    # we apply to 3d convolutions with relu activation
    def _conv_block(self, in_channels, out_channels):
        return nn.Sequential(nn.Conv3d(in_channels, out_channels, kernel_size=3, padding=1),
                             nn.ReLU(),
                             nn.Conv3d(out_channels, out_channels, kernel_size=3, padding=1),
                             nn.ReLU())       


    # upsampling via transposed 3d convolutions
    def _upsampler(self, in_channels, out_channels):
        return nn.ConvTranspose3d(in_channels, out_channels,
                                kernel_size=2, stride=2)
    
    def __init__(self, in_channels=1, out_channels=1, 
                 final_activation=None):  #nn.Sigmoid): #None):
        super().__init__()
        
        # the depth (= number of encoder / decoder levels) is
        # hard-coded to 4
        self.depth = 4

        # the final activation must either be None or a Module
        if final_activation is not None:
            assert isinstance(final_activation, nn.Module), "Activation must be torch module"
        
        # all lists of conv layers (or other nn.Modules with parameters) must be wraped
        # itnto a nn.ModuleList
        
        # modules of the encoder path
        self.encoder = nn.ModuleList([self._conv_block(in_channels, 16),
                                      self._conv_block(16, 32),
                                      self._conv_block(32, 64),
                                      self._conv_block(64, 128)])
        # the base convolution block
        self.base = self._conv_block(128, 256)
        # modules of the decoder path
        self.decoder = nn.ModuleList([self._conv_block(256, 128),
                                      self._conv_block(128, 64),
                                      self._conv_block(64, 32),
                                      self._conv_block(32, 16)])
        
        # the pooling layers; we use 2x2 MaxPooling
        self.poolers = nn.ModuleList([nn.MaxPool3d(2) for _ in range(self.depth)])
        # the upsampling layers
        self.upsamplers = nn.ModuleList([self._upsampler(256, 128),
                                         self._upsampler(128, 64),
                                         self._upsampler(64, 32),
                                         self._upsampler(32, 16)])
        # output conv and activation
        # the output conv is not followed by a non-linearity, because we apply
        # activation afterwards
        self.out_conv = nn.Conv3d(16, out_channels, 1)
        self.activation = final_activation
    
    def forward(self, input):
        x = input
        # apply encoder path
        encoder_out = []
        for level in range(self.depth):
            x = self.encoder[level](x)
            encoder_out.append(x)
            x = self.poolers[level](x)

        # apply base
        x = self.base(x)
        
        # apply decoder path
        encoder_out = encoder_out[::-1]
        for level in range(self.depth):
            x = self.upsamplers[level](x)
            x = self.decoder[level](torch.cat((x, encoder_out[level]), dim=1))
        
        # apply output conv and activation (if given)
        x = self.out_conv(x)
        if self.activation is not None:
            x = self.activation(x)
        #x = nn.Sigmoid(x)
        return x

In [6]:
# sorensen dice coefficient implemented in torch
# the coefficient takes values in [0, 1], where 0 is
# the worst score, 1 is the best score
class DiceCoefficient(nn.Module):
    def __init__(self, eps=1e-6):
        super().__init__()
        self.eps = eps
        
    # the dice coefficient of two sets represented as vectors a, b ca be 
    # computed as (2 *|a b| / (a^2 + b^2))
    def forward(self, prediction, target):
        intersection = (prediction * target).sum()
        denominator = (prediction * prediction).sum() + (target * target).sum()
        return (2 * intersection / denominator.clamp(min=self.eps))

In [7]:
# check if we have  a gpu
if torch.cuda.is_available():
    print("GPU is available")
    device = torch.device("cuda")
else:
    print("GPU is not available")
    device = torch.device("cpu")

GPU is available


In [13]:
model = UNet(1, 4)  #, final_activation=nn.Sigmoid()) #second '1' changed to 4
model = model.to(device)
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=1.e-3)
metric = DiceCoefficient()

In [8]:
# start a tensorboard writer
tb_logger = SummaryWriter('runs/Unet')
%tensorboard --logdir runs  --port 6556

In [8]:
# apply training for one epoch
def train(model, loader, loss_function, optimizer, device,
          epoch, log_interval=100, log_image_interval=20, tb_logger=None):
    """ Train model for one epoch.

    Parameters:
    model - the model we are training
    loader - the data loader that provides the training data
        (= pairs of images and labels)
    loss_function - the loss function that will be optimized
    optimizer - the optimizer that is used to update the network parameters
        by backpropagation of the loss
    device - the device used for training. this can either be the cpu or gpu
    epoch - which trainin eppch are we in? we keep track of this for logging
    tb_logger - the tensorboard logger, it is used to communicate with tensorboard
    log_image_interval - how often do we send images to tensborboard?
    """

    # set the model to train mode TODO: YOUR CODE HERE
    model.train()
    #n_batches = len(loader)

    # iterate over the batches of this epoch
    for batch_id, (x, y) in enumerate(loader):
        # move input and target to the active device (either cpu or gpu)
        x, y = x.to(device), y.to(device)
        
        # zero the gradients for this iteration TODO: YOUR CODE HERE
        optimizer.zero_grad()
        
        # apply model, calculate loss and run backwards pass TODO: YOUR CODE HERE
        prediction = model(x)
        #prediction = torch.argmax(prediction, dim = 1)
        #prediction = prediction.float()
        #print(prediction.shape)
        #print(torch.unique(prediction))
        #print(torch.unique(x))
        #print(torch.unique(y))
        #print(y[:, 0].shape)
        #prediction = prediction.long()
        y = y.long()
        #loss_value = loss_function(prediction, y)
        loss_value = loss_function(prediction, y[:, 0])
        loss_value.backward()
        optimizer.step()
        
        # log to console
        if batch_id % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                  epoch, batch_id * len(x),
                  len(loader.dataset),
                  100. * batch_id / len(loader), loss_value.item()))

       # log to tensorboard
        if tb_logger is not None:
            step = epoch * len(loader) + batch_id
            tb_logger.add_scalar(tag='train_loss', scalar_value=loss_value.item(), global_step=step)
            # check if we log images in this iteration
            if step % log_image_interval == 0:
                tb_logger.add_images(tag='input', img_tensor=x[:,:,50,:,:].to('cpu'), global_step=step)
                tb_logger.add_images(tag='target', img_tensor=y[:,:,50,:,:].to('cpu'), global_step=step)
                #tb_logger.add_images(tag='prediction', img_tensor=prediction[:,1,50,:,:].to('cpu').detach(), global_step=step)

In [9]:
# run validation after training epoch
def validate(model, loader, loss_function, metric, step=None, tb_logger=None):
    # set model to eval mode TODO: YOUR CODE HERE
    model.eval()
    #n_batches = len(loader)
    
    # running loss and metric values
    val_loss = 0
    val_metric = 0
    
    # disable gradients during validation
    with torch.no_grad():
        
        # iterate over validation loader and update loss and metric values
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            #TODO: YOUR CODE HERE
            prediction = model(x)
            y = y.long()
            #mean_loss += loss_function(prediction, y[:, 0]).item()
            val_loss += loss_function(prediction, y[:,0]).item()
            val_metric += metric(prediction, y).item()
            
            #prediction = prediction.max(1, keepdim=True)[1]
            #predictions.append(prediction[:, 0].to('cpu').numpy())
            #labels.append(y[:, 0].to('cpu').numpy())
        #predictions = np.concatenate(predictions)
        #labels = np.concatenate(labels)
        
    # normalize loss and metric
    val_loss /= len(loader)
    val_metric /= len(loader)
    
    if tb_logger is not None:
        assert step is not None, "Need to know the current step to log validation results"
        tb_logger.add_scalar(tag='val_loss', scalar_value=val_loss, global_step=step)
        tb_logger.add_scalar(tag='val_metric', scalar_value=val_metric, global_step=step)
        # we always log the last validation images
        tb_logger.add_images(tag='val_input', img_tensor=x[:,:,50,:,:].to('cpu'), global_step=step)
        tb_logger.add_images(tag='val_target', img_tensor=y[:,:,50,:,:].to('cpu'), global_step=step)
        #tb_logger.add_images(tag='val_prediction', img_tensor=prediction[...,50,:,:].to('cpu'), global_step=step)
        
    print('\nValidate: Average loss: {:.4f}, Average Metric: {:.4f}\n'.format(val_loss, val_metric))

In [11]:
# during the training you can inspect the 
# predictions in the tensorboard
n_epochs = 100
for epoch in range(n_epochs):
    # train TODO: YOUR CODE HERE add save torch.save(net, "model.pt")
    train(model, train_loader, loss_function, optimizer,
                device, epoch, tb_logger=tb_logger)
    step = (epoch +1) * len(train_loader.dataset)
    # validate TODO: YOUR CODE HERE
    validate(model, val_loader, loss_function,
                   metric, step,
                   tb_logger=tb_logger)
    torch.save(model, "model_{}.pt".format(epoch))

Train Epoch: 0 [0/57 (0%)]	Loss: 1.859687

Validate: Average loss: 0.5864, Average Metric: 0.0005

Train Epoch: 1 [0/57 (0%)]	Loss: 0.679570

Validate: Average loss: 0.1935, Average Metric: 0.0000

Train Epoch: 2 [0/57 (0%)]	Loss: 0.297269

Validate: Average loss: 0.1924, Average Metric: 0.0000

Train Epoch: 3 [0/57 (0%)]	Loss: 0.227704

Validate: Average loss: 0.1777, Average Metric: -0.0001

Train Epoch: 4 [0/57 (0%)]	Loss: 0.136207

Validate: Average loss: 0.1739, Average Metric: -0.0001

Train Epoch: 5 [0/57 (0%)]	Loss: 0.177627

Validate: Average loss: 0.1694, Average Metric: -0.0004

Train Epoch: 6 [0/57 (0%)]	Loss: 0.153932

Validate: Average loss: 0.1781, Average Metric: -0.0001

Train Epoch: 7 [0/57 (0%)]	Loss: 0.265189

Validate: Average loss: 0.1403, Average Metric: -0.0002

Train Epoch: 8 [0/57 (0%)]	Loss: 0.105766

Validate: Average loss: 0.1490, Average Metric: -0.0001

Train Epoch: 9 [0/57 (0%)]	Loss: 0.141511

Validate: Average loss: 0.1277, Average Metric: -0.0005

Tra

QStandardPaths: XDG_RUNTIME_DIR points to non-existing path '/run/user/1043', please create it with 0700 permissions.


In [14]:

#model = model()
#state = torch.load('./model.pt')
#model.load_state_dict(state)

model = torch.load('./model.pt')
validate(model, test_loader, loss_function,
                   metric, step,
                   tb_logger=tb_logger)


NameError: name 'step' is not defined

In [11]:
test_data = DrosoMicroCT('./Project5_Drosophila_microCT_segmentation/resize2/test')
test_loader = DataLoader(test_data, batch_size=2)

In [47]:
testimage, testlabels = test_data[1]

In [48]:
testimage = testimage.to(device)

In [49]:
prediction = model(testimage)

RuntimeError: Expected 5-dimensional input for 5-dimensional weight [16, 1, 3, 3, 3], but got 4-dimensional input of size [1, 128, 128, 128] instead

In [23]:
testimage, testlabel=iter(test_loader).next()
testimage = testimage.to(device)
testlabel = testlabel.long()
prediction = model(testimage)
predictimage = torch.argmax(prediction, dim=1)
predictimage=predictimage.to('cpu')
predictimage=predictimage.detach().numpy()
np.unique(predictimage)
testimage = testimage.to('cpu')
viewer = napari.view_image(testimage)
viewer.add_labels(predictimage)
viewer.add_labels(y)


<Labels layer 'y' at 0x7fbc6014f460>